In [1]:
import sys
import os

assert len(sys.argv) == 3, 'Args are wrong.'

input_path = "./models/v2-1_512-ema-pruned.ckpt"
output_path = "./models/control_sd21_ini.ckpt"

assert os.path.exists(input_path), 'Input model does not exist.'
assert not os.path.exists(output_path), 'Output filename already exists.'
assert os.path.exists(os.path.dirname(output_path)), 'Output path is not valid.'

import torch
from share import *
from cldm.model import create_model


/home/tj/anaconda3/envs/control/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1)
    Python  3.10.11 (you have 3.10.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


logging improved.


In [5]:
def get_node_name(name, parent_name):
    if len(name) <= len(parent_name):
        return False, ''
    p = name[:len(parent_name)]
    if p != parent_name:
        return False, ''
    return True, name[len(parent_name):]


model = create_model(config_path='./models/cldm_v21_reduced.yaml')

ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 32, context_dim is None and using 2 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 32, context_dim is 16 and using 2 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 64, context_dim is None and using 4 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 64, context_dim is 16 and using 4 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 128, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 128, context_dim is 16 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 128, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 128, context_dim is 16 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 128, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 128, context_dim is 16 

In [3]:
pretrained_weights = torch.load(input_path)
if 'state_dict' in pretrained_weights:
    pretrained_weights = pretrained_weights['state_dict']

In [6]:
scratch_dict = model.state_dict()

In [7]:
scratch_dict.keys()

odict_keys(['betas', 'alphas_cumprod', 'alphas_cumprod_prev', 'sqrt_alphas_cumprod', 'sqrt_one_minus_alphas_cumprod', 'log_one_minus_alphas_cumprod', 'sqrt_recip_alphas_cumprod', 'sqrt_recipm1_alphas_cumprod', 'posterior_variance', 'posterior_log_variance_clipped', 'posterior_mean_coef1', 'posterior_mean_coef2', 'logvar', 'model.diffusion_model.time_embed.0.weight', 'model.diffusion_model.time_embed.0.bias', 'model.diffusion_model.time_embed.2.weight', 'model.diffusion_model.time_embed.2.bias', 'model.diffusion_model.input_blocks.0.0.weight', 'model.diffusion_model.input_blocks.0.0.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.0.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.0.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.2.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.2.bias', 'model.diffusion_model.input_blocks.1.0.emb_layers.1.weight', 'model.diffusion_model.input_blocks.1.0.emb_layers.1.bias', 'model.diffusion_model.input_blocks.1.0.ou

In [13]:
type(scratch_dict['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_v.weight'][0][0].item())

float

In [5]:
del model

In [ ]:
target_dict = {}
for k in scratch_dict.keys():
    is_control, name = get_node_name(k, 'control_')
    if is_control:
        copy_k = 'model.diffusion_' + name
    else:
        copy_k = k
    if copy_k in pretrained_weights:
        target_dict[k] = pretrained_weights[copy_k].clone()
    else:
        target_dict[k] = scratch_dict[k].clone()
        print(f'These weights are newly added: {k}')

These weights are newly added: logvar
These weights are newly added: model.diffusion_model.input_blocks.2.0.op.weight
These weights are newly added: model.diffusion_model.input_blocks.2.0.op.bias
These weights are newly added: model.diffusion_model.input_blocks.3.0.in_layers.0.weight
These weights are newly added: model.diffusion_model.input_blocks.3.0.in_layers.0.bias
These weights are newly added: model.diffusion_model.input_blocks.3.0.in_layers.2.weight
These weights are newly added: model.diffusion_model.input_blocks.3.0.in_layers.2.bias
These weights are newly added: model.diffusion_model.input_blocks.3.0.emb_layers.1.weight
These weights are newly added: model.diffusion_model.input_blocks.3.0.emb_layers.1.bias
These weights are newly added: model.diffusion_model.input_blocks.3.0.out_layers.0.weight
These weights are newly added: model.diffusion_model.input_blocks.3.0.out_layers.0.bias
These weights are newly added: model.diffusion_model.input_blocks.3.0.out_layers.3.weight
These 

In [6]:
del pretrained_weights

In [ ]:
model = create_model(config_path='./models/cldm_v21_reduced.yaml')
model.load_state_dict(target_dict, strict=True)
torch.save(model.state_dict(), output_path)
print('Done.')